Packages:

In [1]:
%run stdPackages.ipynb
from pyDbs import read, readSets, broadcast
from py.mBasicInt_NonLinear import *

Read data:

In [2]:
kwargs = {'variables': ['Fundamentals', 'Load', 'Generators_Other'], 
        'variable2D': ['Generators_FuelMix','HourlyVariation'],
        'scalars': ['Scalars'],
        'maps': ['Generators_Categories']}
db = read.dbFromWB(os.path.join(d['data'],'mBasicInt_NonLinear.xlsx'), kwargs)
db.updateAlias(alias=[('h','h_alias')])
readSets(db)

Initialize model:

In [3]:
m = mSimpleNonLinear(db)

Temporary price vector:

In [4]:
p0 = pd.Series([50,50,50,50],index=db['h'],name='p')

Check aggregate supply:

In [5]:
m.fsupply(p0)

h
1     174.98265
2    157.482672
3    148.732684
4    167.982659
dtype: object

Check demand:

In [6]:
m.db['HourlyDemand'].groupby('h').sum()

h
1     40.0
2    120.0
3    200.0
4     40.0
dtype: object

Check excess demand:

In [7]:
m.fexcess_demand(p0)

h
1    -134.98265
2    -37.482672
3     51.267316
4   -127.982659
dtype: object

Find equilibrium price:

In [8]:
m.solve(p0,jacobian='analytical')
m.db['p']

h
1      0.959296
2     17.872620
3    114.549430
4      1.597060
Name: p, dtype: float64

Check equilibrium:

In [9]:
m.fexcess_demand(m.db['p'])

h
1    0.0
2    0.0
3    0.0
4    0.0
dtype: object

In [10]:
m.fsupply(m.db['p'])

h
1     40.0
2    120.0
3    200.0
4     40.0
dtype: object

In [11]:
m.update_db()

In [12]:
m.db['marginalEconomicValue']

id
id1    106.069779
id2     97.991551
id3     81.666458
id4     20.786768
Name: 0, dtype: object

In [13]:
m.db['marginalEconomicRevenue']

id
EDP     33.546619
id1    131.057603
id2    129.322405
id3    125.434076
id4      37.93141
dtype: object

In [14]:
m.db['marginalEconomicCosts']

In [16]:
m.db['ConsumerSurplus']

c   id 
c1  EDP    23163.534699
    id1    23163.534699
    id2    23163.534699
    id3    23163.534699
    id4    23163.534699
dtype: object

In [15]:
m.db['Welfare']

116124.18805087442